<a href="https://colab.research.google.com/github/sumeyyedemir5/nlp-preprocessing_and_textRepresentation/blob/main/Word_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## WORD EMBEDDİNG

kelimeleri sayısal vektörlere dönüştürür. bu temsiller kelimeler arasındaki anlamsal ilişkileri yakalamayı hedefler.

* **word2Vec** : Google tarafınfan geliştirilen ,kelimeleri vektörlere dönüştüren ve bu vektörleri dildeki ilişkileri yakalayacak şekilde eğiten modeldir.
* **GloVe** : kelime gömme temsillerini kelime ortaklıklarını yakalayacak şekilde hesaplayan bir modeldir.
* **FastText** : Facebook tarafından geliştirilen ve kelime gömme temsillerini alt birimlerini de dikkate alarak hesaplayan bir modeldir

In [ ]:
!pip install gensim

In [ ]:
import pandas as pd
from gensim.models import Word2Vec, FastText
from gensim.utils import simple_preprocess #büyük küçük harf çevrimi ve tokenizasyon
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

sentences = [
    "kedi çok tatlı bir hayvandır",
    "köpekler de tatlıdır"
]

tokenized_sentences = [simple_preprocess(sentence) for sentence in sentences]
tokenized_sentences


In [ ]:
word2vec = Word2Vec(sentences=tokenized_sentences,vector_size=50,window=5,min_count=1,sg=0)
#vector_size = her kelime için oluşturulacak vektörlerin boyutunu belirler.boyut arttıkça yorumlama gücü artar
#window = kelimeye en yakın kaç kelime dikkate alınacak?
#min_count = 1 kelimeden sadece 1 adet bulunuyorsa o kelimenin dikkate alınmayacağı anlamına gelir.
#sg=0 BoW algoritmasını kullanacak sg=1 ngram algoritması

fasttext = FastText(sentences=tokenized_sentences,vector_size=50,window=5,min_count=1,sg=0)

def plot_word_embadding(model,title):
  word_vectors = model.wv # modelin öğrendiği tüm kelimeleri ve bunlara karşılık gelen sayısal vektörleri tutan sözlüktür
  words = list(word_vectors.index_to_key)[:1000]
  vectors = [word_vectors[word] for word in words]

  #PCA
  pca = PCA(n_components=3)
  reduced_vectors = pca.fit_transform(vectors)

  #3d görselleştirme
  fig = plt.figure(figsize = (12,8))
  ax = fig.add_subplot(111, projection="3d")

  #vektorler
  ax.scatter(reduced_vectors[:,0],reduced_vectors[:,1],reduced_vectors[:,2])
  #kelimeleri etiketler
  for i, word in enumerate(words):
    ax.text(reduced_vectors[i,0],reduced_vectors[i,1],reduced_vectors[i,2],word, fontsize=12)

  ax.set_title(title)
  ax.set_xlabel("Component 1")
  ax.set_ylabel("Component 2")
  ax.set_zlabel("Component 3")
  plt.show()

plot_word_embadding(word2vec,"Word2Vec")
plot_word_embadding(fasttext,"FastText")

In [ ]:
from sklearn.cluster import KMeans
import re

In [ ]:
df= pd.read_csv("/content/drive/MyDrive/IMDB Dataset.csv")

In [ ]:
documents = df["review"]

In [ ]:
#metin temizleme
def clean_text(text):
  text = re.sub(r"\d+","",text)
  text = re.sub(r"[^\w\s]","",text)
  text = " ".join([word for word in text.split() if len(word) > 2])
  return text

In [ ]:
cleaned_doc = [clean_text(doc) for doc in documents]
#tokenizasyon
tokenized_documents = [simple_preprocess(doc) for doc in cleaned_doc]

In [ ]:
model = Word2Vec(sentences = tokenized_documents, vector_size = 50,window =5, min_count=1, sg=0)
word_vectors = model.wv

words = list(word_vectors.index_to_key)[:500]
vectors = [word_vectors[word] for word in words]

In [ ]:
kmeans = KMeans(n_clusters= 3) #3 küme
kmeans.fit(vectors)
clusters = kmeans.labels_ #her kelimenin hangi kümeye ait olduğunu gösterir. örn: [0,0,1,1,2..]

pca = PCA(n_components = 2) #analizi kolaylaştırmak için 50 boyutu 2 boyuta indiriyoruz.
reduced_vectors = pca.fit_transform(vectors)

In [ ]:
#2d görselleştirme
plt.figure()
plt.scatter(reduced_vectors[:,0],reduced_vectors[:,1],c = clusters,cmap ="viridis" )
centers = pca.transform(kmeans.cluster_centers_)
plt.scatter(centers[:,0], centers[:,1], c="red", marker ="x", s=80, label = "merkez")
plt.legend()

for i, word in enumerate(words):
  plt.text(reduced_vectors[i,0], reduced_vectors[i,1], word, fontsize=8)
plt.title("word2Vec")